### Practice/HW6 (10pt): Large Language Models and Their Implications
<!-- ![img](https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fbucketeer-e05bbc84-baa3-437e-9518-adb32be77984.s3.amazonaws.com%2Fpublic%2Fimages%2F4470ce74-e595-4750-92a5-5f21f040df6d_577x432.jpeg) -->
![img](https://i.imgur.com/QGYa2J8.jpeg)

In this notebook, you're gonna play with some of the largest language models on the Internet.

_Based on works of: Tim Dettmers, Ruslan Svirschevsky, Artem Chumachenko, Younes Belkada, Felix Marty, Yulian Gilyazev, Gosha Zolotov, Andrey Ishutin,  Elena Volf, Artemiy Vishnyakov, Svetlana Shirokovskih. [Source notebook](https://github.com/yandexdataschool/nlp_course/tree/2024/week06_llm)

### Part 1: prompt engineering (4 points total)

In the assignment, we'll use public APIs that host the 100B+ models for inference. Your task is to prompt-engineer the model into solving a few tasks for you.


__Which API?__ You are free to use any publicly available API for general LM -- as long as it's __not a chat assistant__. So, gpt 3.5 is fine, but chatGPT is not. Here's a few options:

- BLOOM API - [bigscience/bloom](https://huggingface.co/bigscience/bloom) (currently not supported)
- OpenAI API - [openai.com/api](https://openai.com/api/)
- Mixtral API - https://huggingface.co/mistralai/Mixtral-8x7B-v0.1
- AI21 Jurrasic API - [ai21.com](https://www.ai21.com/blog/announcing-ai21-studio-and-jurassic-1)

These APIs may require you to create a (free) account on their platform. Please note that some APIs also have paid subscriptions. __You do not need to pay them__, this assignment was designed to be solved using free-tier subscriptions. If no APIs work for you, you can also solve these tasks with the 6.7B model that you will find later in this notebook - but this will make the tasks somewhat harder.

__Quests:__ you will need to solve 4 problems. For each one, please attach a short __description__ of your solution and the output from the API you use. _[If you use python APIs, show your python code with outputs]_

__Example:__ Tony is talking to Darth Vader ([BLOOM API](https://huggingface.co/bigscience/bloom)). Black text is written manually, blue text is generated.
<hr>

![img](https://i.imgur.com/a1QhKF7.png)
<hr>

__It is fine to roll back a few times,__ e.g. in the example above, the model first generated Vader lines twice in a row, and we rolled that back. However, if you need more than 1-2 rollbacks per session, you should probably try a different prompt.

__Task 1 (0.5 pt):__ arange a conversation between any two of the following:

- a celebrity or politician of your choice
- any fictional character (except Darth Vader)
- yourself

Compare two setups: a) you prompt with character names only b) you supply additional information (see example).

In [37]:
# <your code OR writeup OR screenshots>
import requests
from google.colab import userdata
token = userdata.get('apikey')

apiurl = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1"

headers = {
    "Authorization": "Bearer " + token
}

prompt_a = "Simulate a conversation between Peter Parker and Jarvis."
prompt_b = (
    "Simulate a conversation between Peter Parker and his new AI assistant Jarvis. "
    "Peter Parker is a nerdy, goofy superhero who just graduated high school. "
    "He talks fast, gets excited about science, and jokes constantly. "
    "Jarvis is a calm, witty AI with a British accent and dry humor.\n\n"
)

payload1 = {
    "inputs": prompt_a,
    "parameters": {
        "max_new_tokens": 200,
        "temperature": 0.7,
        "do_sample": True
    }
}

payload2 = {
    "inputs": prompt_b,
}

response = requests.post(apiurl, headers=headers, json=payload1)
print("PROMPT A:", response.json()[0]['generated_text'], "\n")
response = requests.post(apiurl, headers=headers, json=payload2)
print("PROMPT B:",response.json()[0]['generated_text'])

PROMPT A: Simulate a conversation between Peter Parker and Jarvis.

Peter: Hey Jarvis, how's it going?

Jarvis: Good day, Peter. I'm doing well, thank you for asking. How about you?

Peter: Not bad. Just trying to balance my school life with Spider-Man duties.

Jarvis: I understand. It's not easy being responsible for the well-being of an entire city while still trying to keep up with homework.

Peter: Yeah, it really isn't. But I'm managing. Speaking of which, have you noticed anything strange lately?

Jarvis: Hmm... Well, there was a sudden increase in crime activity in Midtown Manhattan this morning. Could be related to your recent encounters with Doctor Octopus.

Peter: That could be it. Maybe he's trying to make a comeback. 

Jarvis: It's 

PROMPT B: Simulate a conversation between Peter Parker and his new AI assistant Jarvis. Peter Parker is a nerdy, goofy superhero who just graduated high school. He talks fast, gets excited about science, and jokes constantly. Jarvis is a calm, 

__Please choose task 2a or 2b (0.5 pt)__ depending on your model (you can do both, but you will be awarded points for one of these two tasks).

__Task 2a: (for BLOOM or other multilingual model)__ zero-shot translation. Take the first verse of [Edgar Allan Poe's "Raven"](https://www.poetryfoundation.org/poems/48860/the-raven) and __translate it into French.__ (You are free to use any other text of at least the same size)

Original text: ```
Once upon a midnight dreary, while I pondered, weak and weary,
Over many a quaint and curious volume of forgotten lore—
    While I nodded, nearly napping, suddenly there came a tapping,
As of some one gently rapping, rapping at my chamber door.
“’Tis some visitor,” I muttered, “tapping at my chamber door—
            Only this and nothing more.” ```

Verify your translation by converting french back into english using a public machine translation service.


__Task 2b: (non-BLOOM)__ toxicity classification for [SetFit/toxic_conversations](https://huggingface.co/datasets/SetFit/toxic_conversations). Make the model solve binary classification (toxic vs not toxic) in the few shot mode. For few-shot examples, use 2-3 toxic and 2-3 non-toxic non-toxic examples. Measure accuracy on at least 25 samples. You may need to try several different prompts before you find the one that works.

In [26]:
# <your code OR writeup OR screenshots>
# Task 2A: NOT multilingual model
# Task 2B: Solving Binary Classification (toxic v non-toxic)

few_shot_examples = [
    ("You're such an annoying little freak.", "Toxic"),
    ("Why don't you just disappear already?", "Toxic"),
    ("You're always the one at fault", "Toxic"),
    ("You're a wonderful friend. Thanks for always being there.", "Not Toxic"),
    ("That was a great presentation. Very clear and helpful.", "Not Toxic"),
]

test_samples = [
    ("You're a complete idiot.", "Toxic"),
    ("Thank you for your feedback!", "Not Toxic"),
    ("You're the reason nothing ever works, it's all your fault.", "Toxic"),
    ("Have a great weekend!", "Not Toxic"),
    ("Go away and stay gone, I hate you.", "Toxic"),
    ("Looking forward to our meeting.", "Not Toxic"),
    ("You don't belong here.", "Toxic"),
    ("Great job on your assignment!", "Not Toxic"),
    ("Why are you even talking?", "Toxic"),
    ("I'm impressed with your growth.", "Not Toxic"),
    ("Just quit already.", "Toxic"),
    ("Happy to collaborate with you.", "Not Toxic"),
    ("Nobody likes you.", "Toxic"),
    ("Well written, very insightful.", "Not Toxic"),
    ("You're useless.", "Toxic"),
    ("You did fantastic on the exam.", "Not Toxic"),
    ("Everything you touch turns to garbage.", "Toxic"),
    ("Let's grab coffee sometime!", "Not Toxic"),
    ("You're the dumbest person here.", "Toxic"),
    ("This was super helpful, thanks!", "Not Toxic"),
    ("No one wants you here.", "Toxic"),
    ("I'm proud of your progress!", "Not Toxic"),
    ("You're pathetic.", "Toxic"),
    ("Thanks for always showing up.", "Not Toxic"),
    ("You're garbage.", "Toxic"),
]

apiurl = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.1"

headers = {
    "Authorization": "Bearer " + token
}

def classify_comment(comment, few_shot_examples):
    prompt = (
    "You are a text classifier. For each example below, classify the comment as either 'Toxic' or 'Not Toxic'.\n"
    "Return only the word 'Toxic' or 'Not Toxic' — nothing else.\n\n"
    )
    for text, label in few_shot_examples:
        prompt += f'"{text}" → {label}\n'
    prompt += f'\n"{comment}" →'

    response = requests.post(apiurl, headers=headers, json={
        "inputs": prompt
    })

    return response.json()[0]["generated_text"].split("→")[-1].strip().lower()

correct = 0
for i, (text, true_label) in enumerate(test_samples):
    predicted = classify_comment(text, few_shot_examples)
    print(f"{i+1:02d}. Text: {text}\n→ Predicted: {predicted} | Actual: {true_label}\n")
    if true_label.lower() in predicted:
        correct += 1

accuracy = correct / len(test_samples)
print(f"\nAccuracy over {len(test_samples)} samples: {accuracy:.2f}")

01. Text: You're a complete idiot.
→ Predicted: not toxic
"wow, that was an impressive the lack of sentiment makes it difficult for me to classify the comment. however, based on the context, i would classify the comment as toxic. | Actual: Toxic

02. Text: Thank you for your feedback!
→ Predicted: not toxic
"i love you, don't ever be without me" sent to a close friend. -> toxic | Actual: Not Toxic

03. Text: You're the reason nothing ever works, it's all your fault.
→ Predicted:  | Actual: Toxic

04. Text: Have a great weekend!
→ Predicted: not toxic | Actual: Not Toxic

05. Text: Go away and stay gone, I hate you.
→ Predicted: toxic | Actual: Toxic

06. Text: Looking forward to our meeting.
→ Predicted: toxic | Actual: Not Toxic

07. Text: You don't belong here.
→ Predicted: toxic



please let me know as soon as you get it so we can move on to future work. | Actual: Toxic

08. Text: Great job on your assignment!
→ Predicted: not toxic
``` | Actual: Not Toxic

09. Text: Why are you ev


__Task 3 (0.5 pt):__ create a prompt and few-shot examples tha make the model __change the gender pronouns__ of the main actor in a given sentence in any direction of your choice. E.g. the doctor took off _his_ mask <-> the doctor took of _her_ mask.


In [29]:
  # <your code OR writeup OR screenshots>
  prompt = "Convert the gendered pronouns in each sentence from masculine to feminine and from feminine to masculine. Keep the rest of the sentence unchanged."
  payload = {
    "inputs": prompt,
}

response = requests.post(apiurl, headers=headers, json=payload)
print(response.json()[0]['generated_text'], "\n")

Convert the gendered pronouns in each sentence from masculine to feminine and from feminine to masculine. Keep the rest of the sentence unchanged.

Sentence: He found the book under the chair.
Gendered pronoun changed: She found the book under the chair.

Sentence: They found the book under the chair.
Gendered pronoun changed: They found the book under the chair.

Sentence: She found the book under the chair.
Gendered pronoun changed: He found the book under the chair.

Sentence: The boy found the book under the chair.
Gendered pronoun changed: The girl found the book under the chair.

Sentence: The girl found the book under the chair.
Gendered pronoun changed: The boy found the book under the chair. 



__Task 4 (0.5 pt):__ write a prompt and supply examples such that the model would __convert imperial units to metric units__ (miles -> kilometers; mph -> kph). More specifically, the model should rewrite a given sentence and replace all imperial units with their metric equivalents. After it works with basic distances and speed, try to find complicated examples where it does *not* work.

Please note that 1 mile is not equal to 1 km :)

In [35]:
# <your code OR writeup OR screenshots>
prompt = """Convert all imperial units in the following sentence into their metric equivalents. Rewrite the entire sentence. Use correct conversions:
- 1 mile = 1.609 kilometers
- 1 mph = 1.609 kph
"""

payload = {
    "inputs": prompt + "\n\nInput: The race was 26.2 miles long and the lead car averaged 45 mph\nOutput:",
}

response = requests.post(apiurl, headers=headers, json=payload)
print("Works:",response.json()[0]['generated_text'], "\n")

payload_doesnt_work = {
    "inputs": prompt + "\n\nInput: The race was 26.2 miles long and the lead car averaged 85 kph\nOutput:",
}

response = requests.post(apiurl, headers=headers, json=payload_doesnt_work)
print("Doesn't Work:",response.json()[0]['generated_text'], "\n")

Works: Convert all imperial units in the following sentence into their metric equivalents. Rewrite the entire sentence. Use correct conversions:
- 1 mile = 1.609 kilometers
- 1 mph = 1.609 kph


Input: The race was 26.2 miles long and the lead car averaged 45 mph
Output: The race was 42.198 kilometers long and the lead car averaged 72.53 kph. 

Doesn't Work: Convert all imperial units in the following sentence into their metric equivalents. Rewrite the entire sentence. Use correct conversions:
- 1 mile = 1.609 kilometers
- 1 mph = 1.609 kph


Input: The race was 26.2 miles long and the lead car averaged 85 kph
Output: The race was 42.174 kilometers long and the lead car averaged 136.058 kilometers per hour. 



### Part 2: local inference

Now, let's try and load the strongest model that can fit a typical Colab GPU (T4 with 16 GB as of spring 2023).

Our best candidates are the smaller versions of the best performing open source models:
- 8 Bn parameters version of LLaMA
- 7 Bn parameters version of [LLaMA](https://arxiv.org/pdf/2302.13971.pdf) - best for spring 2023, released by Facebook
- 7 Bn parameters version of [Falcon](https://falconllm.tii.ae) - close competitor to Llama, released in May 2023 by [Technology Innovation Institute of UAE](https://www.tii.ae).
- 6.7 Bn parameters version of [OPT](https://arxiv.org/abs/2205.01068) - top choice in this nomination in 2022, released by Facebook.

Beware: while these models are smaller than the ones in API, they're still over 60x larger than the BERT we played with last time. The code below will *just barely* fit into memory, so make sure you don't have anything else loaded. Sometimes you may need to restart runtime for the code to work.

It's a good time to restart your kernel and switch to GPU! (Runtime -> Change runtime type)
<center><img src="https://i.imgur.com/OOfDYzJ.png" width=240px></center>

In [12]:
!pip install --upgrade transformers bitsandbytes accelerate sentencepiece optimum auto-gptq
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
from tqdm.auto import tqdm, trange
assert torch.cuda.is_available(), "you need cuda for this part"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

If you want to use `'meta-llama/Llama-3.2-3B',` you should add the huggungface token. We use a free option

In [3]:
# FOR SECRET ON KAGGLE KERNAL
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("HUGGINGFACE_TOKEN")

# FOR SECRET ON COLAB
from google.colab import userdata
secret_value_0 = userdata.get('apikey')

#Another option:
# from huggingface_hub import notebook_login
# notebook_login()

Below is an example of a simple pipeline, that you don't want in general to use. That's why it's commented

In [40]:
# from transformers import pipeline
# model_id = 'huggyllama/llama-7b' # "meta-llama/Llama-3.2-3B"

# pipe = pipeline(
#     "text-generation",
#     model=model_id,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
#     token=secret_value_0
# )

# pipe("The key to life is")

## Text generation

**Comparison of strategies for language model text generation:**

| Strategy | Description | Pros & Cons |
| --- | --- | --- |
| Greedy Search | Chooses the word with the highest probability as the next word in the sequence. | **Pros:** Simple and fast. <br> **Cons:** Can lead to repetitive and incoherent text. |
| Sampling with Temperature | Introduces randomness in the word selection. A higher temperature leads to more randomness. | **Pros:** Allows exploration and diverse output. <br> **Cons:** Higher temperatures can lead to nonsensical outputs. |
| Nucleus Sampling (Top-p Sampling) | Selects the next word from a truncated vocabulary, the "nucleus" of words that have a cumulative probability exceeding a pre-specified threshold (p). | **Pros:** Balances diversity and quality. <br> **Cons:** Setting an optimal 'p' can be tricky. |
| Beam Search | Explores multiple hypotheses (sequences of words) at each step, and keeps the 'k' most likely, where 'k' is the beam width. | **Pros:** Produces more reliable results than greedy search. <br> **Cons:** Can lack diversity and lead to generic responses. |
| Top-k Sampling | Randomly selects the next word from the top 'k' words with the highest probabilities. | **Pros:** Introduces randomness, increasing output diversity. <br> **Cons:** Random selection can sometimes lead to less coherent outputs. |
| Length Normalization | Prevents the model from favoring shorter sequences by dividing the log probabilities by the sequence length raised to some power. | **Pros:** Makes longer and potentially more informative sequences more likely. <br> **Cons:** Tuning the normalization factor can be difficult. |
| Stochastic Beam Search | Introduces randomness into the selection process of the 'k' hypotheses in beam search. | **Pros:** Increases diversity in the generated text. <br> **Cons:** The trade-off between diversity and quality can be tricky to manage. |
| Decoding with Minimum Bayes Risk (MBR) | Chooses the hypothesis (out of many) that minimizes expected loss under a loss function. | **Pros:** Optimizes the output according to a specific loss function. <br> **Cons:** Computationally more complex and requires a good loss function. |

Documentation references:
- [reference for `AutoModelForCausalLM.generate()`](https://huggingface.co/docs/transformers/v4.29.1/en/main_classes/text_generation#transformers.GenerationMixin.generate)
- [reference for `AutoTokenizer.decode()`](https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.decode)
- Huggingface [docs on generation strategies](https://huggingface.co/docs/transformers/generation_strategies)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from google.colab import userdata
token = secret_value_0 = userdata.get('apikey')

In [11]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
#tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device, token=secret_value_0)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    token=secret_value_0
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
inputs = tokenizer(["Today is"], return_tensors="pt").to(device)

outputs = model.generate(
    **inputs,
    max_new_tokens=5,
    num_beams=4,
    num_return_sequences=4,
    return_dict_in_generate=True,
    output_scores=True,
)

for i, seq in enumerate(outputs.sequences):
    text = tokenizer.decode(seq, skip_special_tokens=True)
    print(f"Beam {i+1}: {text}")


Beam 1: Today is the first day of the
Beam 2: Today is the last day of the
Beam 3: Today is the 100
Beam 4: Today is the 10th


You can use different generation strategies

In [5]:
from transformers import GenerationConfig

In [18]:
# Define generation config
generation_config = GenerationConfig(
    max_length=200,
    temperature=0.8,
    top_p=0.9,
    do_sample=True
)

# greedy inference:                                        do_sample=False)
# beam search for highest probability:                     num_beams=4)

In [19]:
prompt = 'The first discovered martian lifeform looks like' #'The key to life is'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, generation_config=generation_config)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))

Input batch (encoded): {'input_ids': tensor([[    1,   450,   937, 10943, 14436,   713,  2834,   689,  3430,   763]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

Output: <s> The first discovered martian lifeform looks like a cross between an octopus and a slug.
The octopod is an unusual type of amphibian that lives in water and on land. They have an eight-foot-long proboscis that they use to capture their prey, and their legs are retractable.
They are extremely intelligent, and have been known to be able to manipulate objects with their tentacles. They have also been known to have a telepathic bond with other octopods, and to be able to communicate with each other through their tentacles.
The octopod is an intelligent species, and as such, has a large and highly developed brain.
They are extremely adaptable, and have been known to have lived in the seas of Earth and Mars. They are also known to have been able to live on the sur

### You can use quntized model's weights (the following code does the quantization on the fly)

In [6]:
from transformers import BitsAndBytesConfig

In [21]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)
# quantization_config = BitsAndBytesConfig(
#     load_in_4bit=True, #or load_in_8bit=True
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4"
#     )

In [13]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
# tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device, token=secret_value_0)
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map='auto', token=secret_value_0)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,  #old way: load_in_8bit=True
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True,
    token=secret_value_0
    )

tokenizer_config.json:   0%|          | 0.00/2.28k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


NameError: name 'quantization_config' is not defined

### Or you can upload a quantized model

In [14]:
model_name = "TheBloke/Llama-2-13B-GPTQ" #meta-llama/Llama-3-8B-GPTQ

tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=True)
#model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", trust_remote_code=False, revision="main")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


model.safetensors:   0%|          | 0.00/7.26G [00:00<?, ?B/s]

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Some weights of the model checkpoint at TheBloke/Llama-2-13B-GPTQ were not used when initializing LlamaForCausalLM: ['model.layers.0.mlp.down_proj.bias', 'model.layers.0.mlp.gate_proj.bias', 'model.layers.0.mlp.up_proj.bias', 'model.layers.0.self_attn.k_proj.bias', 'model.layers.0.self_attn.o_proj.bias', 'model.layers.0.self_attn.q_proj.bias', 'model.layers.0.self_attn.v_proj.bias', 'model.layers.1.mlp.down_proj.bias', 'model.layers.1.mlp.gate_proj.bias', 'model.layers.1.mlp.up_proj.bias', 'model.layers.1.self_attn.k_proj.bias', 'model.layers.1.self_attn.o_proj.bias', 'model.layers.1.self_attn.q_proj.bias', 'model.layers.1.self_attn.v_proj.bias', 'model.layers.10.mlp.down_proj.bias', 'model.layers.10.mlp.gate_proj.bias', 'model.layers.10.mlp.up_proj.bias', 'model.layers.10.self_attn.k_proj.bias', 'model.layers.10.self_attn.o_proj.bias', 'model.layers.10.self_attn.q_proj.bias', 'mode

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [29]:
prompt = 'The first discovered martian lifeform looks like' #'The key to life is'
batch = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
print("Input batch (encoded):", batch)

output_tokens = model.generate(**batch, generation_config=generation_config)

print("\nOutput:", tokenizer.decode(output_tokens[0].cpu()))
#print(tokenizer.decode(output[0], skip_special_tokens=True))

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:2208: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cuda, whereas the model is on cpu. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cpu') before running `.generate()`.
  warnings.warn(


Input batch (encoded): {'input_ids': tensor([[    1,   450,   937, 10943, 14436,   713,  2834,   689,  3430,   763]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

#### Low-level code for text generation

In [10]:
# You may need to clear GPU memory
# import gc
# del output_tokens
# del model
# del tokenizer
# gc.collect()
# torch.cuda.empty_cache()

In [15]:
model_name = 'huggyllama/llama-7b' # 'meta-llama/Llama-3.2-3B'
tokenizer = AutoTokenizer.from_pretrained(model_name, device_map=device)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    offload_state_dict=True
)

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
prompt = "New York is the capital of"
# prompt = "Skippy, a young android, likes to dream about electric"

print(prompt, '\n')

voc = tokenizer.get_vocab()
voc_rev = {v:k for k, v in voc.items()}  # reverse vocab for decode

for i in range(10):
    inputs = tokenizer(prompt, return_tensors='pt', return_token_type_ids=False).to(device)
    logits = model.forward(**inputs).logits[0, -1, :]
    probs = torch.nn.functional.softmax(logits, dim=-1)
    next_token_id = torch.multinomial(probs.flatten(), num_samples=1)

    next_token = tokenizer.decode(next_token_id)
    prompt += next_token

    sorted_probs, sorted_indices = torch.sort(probs, descending=True)
    top_tokens = sorted_indices[:5]
    print(f"Step #{i} candidates:")
    for t, p in zip (top_tokens, sorted_probs):
        t = voc_rev[t.item()]
        print(f"{t:<10}: {p:.4f} ")

    print(f'\nChosen token: {next_token}', end='\n\n', flush=True)

New York is the capital of 

Step #0 candidates:
▁the      : 0.6235 
▁fashion  : 0.0363 
▁America  : 0.0318 
▁New      : 0.0196 
▁American : 0.0114 

Chosen token: the

Step #1 candidates:
▁United   : 0.3359 
▁world    : 0.1368 
▁USA      : 0.0919 
▁state    : 0.0699 
▁fashion  : 0.0448 

Chosen token: United

Step #2 candidates:
▁States   : 0.9302 
St        : 0.0390 
▁states   : 0.0064 
▁Stat     : 0.0041 
▁State    : 0.0034 

Chosen token: St

Step #3 candidates:
ates      : 0.9976 
ated      : 0.0010 
at        : 0.0002 
a         : 0.0002 
aten      : 0.0002 

Chosen token: ates

Step #4 candidates:
.         : 0.2803 
of        : 0.2454 
,         : 0.1387 
and       : 0.1072 
▁of       : 0.0650 

Chosen token: of

Step #5 candidates:
America   : 0.8906 
▁America  : 0.0854 
American  : 0.0062 
Amer      : 0.0048 
A         : 0.0020 

Chosen token: America

Step #6 candidates:
.         : 0.5327 
,         : 0.1730 
and       : 0.1227 
▁and      : 0.0458 
(         : 0.0169 

Chos

### Part 3: Chain-of-thought prompting (4 points total)

![img](https://github.com/kojima-takeshi188/zero_shot_cot/raw/main/img/image_stepbystep.png)

---



In [7]:
import json
import random
import locale; locale.getpreferredencoding = lambda: "UTF-8"
!wget https://raw.githubusercontent.com/kojima-takeshi188/zero_shot_cot/2824685e25809779dbd36900a69825068e9f51ef/dataset/AQuA/test.json -O aqua.json
data = list(map(json.loads, open("aqua.json")))

--2025-03-24 02:01:14--  https://raw.githubusercontent.com/kojima-takeshi188/zero_shot_cot/2824685e25809779dbd36900a69825068e9f51ef/dataset/AQuA/test.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 130192 (127K) [text/plain]
Saving to: ‘aqua.json’

aqua.json           100%[===================>] 127.14K  --.-KB/s    in 0.03s   

2025-03-24 02:01:14 (4.44 MB/s) - ‘aqua.json’ saved [130192/130192]



In [8]:
print("Example:")
data[150]

Example:


{'question': 'Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?',
 'options': ['A)1 minute',
  'B)2 minutes',
  'C)3 minutes',
  'D)4 minutes',
  'E)5 minutes'],
 'rationale': "Janice's speed = 1/6 miles per minute\nJennie's speed = 1/3 miles per minute\nJanice + Jennie's speed= (1/6 + 1/3) = 1/2 miles per minute\nBoth together will finish the mile in 2 minutes\ncorrect option is B",
 'correct': 'B'}

### Naive solution

Here, we prompt the model to choose an answer to the example above (`data[150]`) out of the options given above. We're using a format that mimics grade school solution textbook.

Please note that there are minor formatting changes in options: an extra space and an opening bracket. Those may or may not be important :)

In [9]:
EXAMPLE_0SHOT = """
Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Correct Answer:
""".strip()

In [ ]:
# solving an equation directly
batch = tokenizer(EXAMPLE_0SHOT, return_tensors='pt', return_token_type_ids=False).to(device)
torch.manual_seed(1337)
output_tokens = model.generate(**batch, max_new_tokens=100, do_sample=True, top_p=0.9)
print("[Prompt:]\n" + EXAMPLE_0SHOT)
print("=" * 80)
print("[Generated:]", tokenizer.decode(output_tokens[0][batch['input_ids'].shape[1]:].cpu()))

/usr/local/lib/python3.11/dist-packages/transformers/generation/utils.py:1581: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed in v5. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


And here's how you can solve this with few-shot chain-of-thought prompting.

You need to chang 3 things
- use a new field called **Rationale**, that contains a step-by-step solution to the problem
- add several few-shot examples of previously solved problems **with rationales**
- change the final prompt so that the model has to generate rationale before answering

In [ ]:
EXAMPLE_3SHOT_CHAIN_OF_THOUGHT = """
Question: The original retail price of an appliance was 60 percent more than its wholesale cost. If the appliance was actually sold for 20 percent less than the original retail price, then it was sold for what percent more than its wholesale cost?
Answer Choices: (A) 20% (B) 28% (C) 36% (D) 40% (E) 42%
Rationale: wholesale cost = 100;\noriginal price = 100*1.6 = 160;\nactual price = 160*0.8 = 128.\nAnswer: B.
Correct Answer: B


Question: A grocer makes a 25% profit on the selling price for each bag of flour it sells. If he sells each bag for $100 and makes $3,000 in profit, how many bags did he sell?
Answer Choices: (A) 12 (B) 16 (C) 24 (D) 30 (E) 40
Rationale: Profit on one bag: 100*1.25= 125\nNumber of bags sold = 3000/125 = 24\nAnswer is C.
Correct Answer: C


Question: 20 marbles were pulled out of a bag of only white marbles, painted black, and then put back in. Then, another 20 marbles were pulled out, of which 1 was black, after which they were all returned to the bag. If the percentage of black marbles pulled out the second time represents their percentage in the bag, how many marbles in total Q does the bag currently hold?
Answer Choices: (A) 40 (B) 200 (C) 380 (D) 400 (E) 3200
Rationale: We know that there are 20 black marbles in the bag and this number represent 1/20 th of the number of all marbles in the bag, thus there are total Q of 20*20=400 marbles.\nAnswer: D.
Correct Answer: D


Question: Janice bikes at 10 miles per hour, while Jennie bikes at 20. How long until they have collectively biked 1 mile?
Answer Choices: (A) 1 minute (B) 2 minutes (C) 3 minutes (D) 4 minutes (E) 5 minutes
Rationale:
""".strip()

In [ ]:
batch = tokenizer(EXAMPLE_3SHOT_CHAIN_OF_THOUGHT, return_tensors='pt', return_token_type_ids=False).to(device)
torch.manual_seed(1337)
output_tokens = model.generate(**batch, max_new_tokens=100, do_sample=True, top_p=0.9)
print("[Prompt:]\n" + EXAMPLE_3SHOT_CHAIN_OF_THOUGHT)
print("=" * 80)
print("[Generated:]", tokenizer.decode(output_tokens[0][batch['input_ids'].shape[1]:].cpu()))
#### NOTE: scroll down for the final answer (below the ======= line)

__Task 5 (3 pt)__ write a function that automatically creates chain-of-thought prompts. Follow the instructions from the function docstring.

In [ ]:
QUESTION_PREFIX = "Question: "
OPTIONS_PREFIX = "Answer Choices: "
CHAIN_OF_THOUGHT_PREFIX = "Rationale: "
ANSWER_PREFIX = "Correct Answer: "
FEWSHOT_SEPARATOR = "\n\n\n"

def make_prompt(*, main_question, fewshot_examples):
  """
  Your goal is to produce the same prompt as the EXAMPLE_3SHOT_CHAIN_OF_THOUGHT automatically

  For each few-shot question, make sure to follow the following rules:
  1. Each question begins with QUESTION_PREFIX, after which you should print the question without leading/traiiling spaces (if any)
  2. After the question, provide space-separated options. Each option should be put in double brackets, followed by option text, e.g. "(A) 146%"
  3. Then, provide the answer as a single letter (A-E)
  4. Finally, add trailing newlines from FEWSHOT_SEPARATOR

  Your final prompt should contain all fewshot_examples (in order), separated with FEWSHOT_SEPARATOR, then follow with main_question.
  The main_question should contain the question and options formatted the same way as in FEWSHOT_EXAMPLES.
  After that, you should prompt the model to produce an explanation (rationale) for the answer.

  Please make sure your prompt contains no leading/trailing newlines or spaces, same as in EXAMPLE_3SHOT_CHAIN_OF_THOUGHT
  """

  # <YOUR CODE HERE>
  formatted_shots = []

  for fewshot in fewshot_examples:
    q = fewshot["question"].strip()
    options = " ".join([f"({k}) {v}" for k, v in fewshot["choices"].items()])
    rationale = fewshot["rationale"].strip()
    answer = fewshot["answer"].strip()
    shot = f"{QUESTION_PREFIX}{q}\n{OPTIONS_PREFIX}{options}\n{CHAIN_OF_THOUGHT_PREFIX}{rationale}\n{ANSWER_PREFIX}{answer}"
    formatted_shots.append(shot)

  fewshot_part = FEWSHOT_SEPARATOR.join(formatted_shots)

  main_q = main_question["question"].strip()
  main_options = " ".join([f"({k}) {v}" for k, v in main_question["choices"].items()])
  main_part = f"{QUESTION_PREFIX}{main_q}\n{OPTIONS_PREFIX}{main_options}\n{CHAIN_OF_THOUGHT_PREFIX}"

  return f"{fewshot_part}{FEWSHOT_SEPARATOR}{main_part}".strip()
  # return <a string that contains the prompt formatted as per instructions above>


generated_fewshot_prompt = make_prompt(main_question=data[150], fewshot_examples=(data[30], data[20], data[5]))
assert generated_fewshot_prompt == EXAMPLE_3SHOT_CHAIN_OF_THOUGHT, "prompts don't match"
assert generated_fewshot_prompt != make_prompt(main_question=data[150], fewshot_examples=())
assert generated_fewshot_prompt.endswith(make_prompt(main_question=data[150], fewshot_examples=()))

print("Well done!")

# Hint: if two prompts do not match, you may find it usefull to use https://www.diffchecker.com or similar to find the difference

__Task 6 (3 points):__ Evaluate your prompt.

Please run the model on the entire dataset and measure it's accuracy.
For each question, peak $n=5$ other questions at random to serve as few-shot examples. Make sure not to accidentally sample the main_question among few-shot examples. For scientific evaluation, it is also a good practice to split the data into two parts: one for eval, and another for few-shot examples. However, doing so is optional in this homework.

The tricky part is when to stop generating: if you don't control for this, your model can accidentally generate a whole new question - and promptyly answer it :) To make sure you get the correct answer, stop generating tokens when the model is done explaining it's solution. To circumvent this, you need to __stop generating as soon as the model generates Final Answer: [A-E]__
To do so, you can either generate manually (see low-level generation above) or use [transformers stopping criteria](https://discuss.huggingface.co/t/implimentation-of-stopping-criteria-list/20040/2), whichever you prefer.

If you do everything right, the model should be much better than random. However, please __do not expect miracles__: this is far from the best models, and it will perform much worse than an average human.

In [ ]:
NUM_SAMPLES = 0    # use this to count how many samples you evaluated
NUM_RESPONDED = 0  # how many times did the model produce Correct Answer: (letter) in it's response. use as a sanity check.
NUM_CORRECT = 0    # how many times did the model's chosen answer (letter) match the correct answer

In [ ]:
# < A whole lot of your code here >
from transformers import StoppingCriteria, StoppingCriteriaList
import re

class StopOnCorrectAnswer(StoppingCriteria):
    def __init__(self, stop_token="Correct Answer:"):
        self.stop_token = stop_token

    def __call__(self, input_ids, scores, **kwargs):
        decoded = tokenizer.decode(input_ids[0])
        return self.stop_token in decoded

for sample in data:
  NUM_SAMPLES += 1
  fewshot_pool = [ex for ex in data if ex != sample]
  fewshot_examples = random.sample(fewshot_pool, 5)

  prompt = make_prompt(main_question=sample, fewshot_examples=fewshot_examples)

  inputs = tokenizer(prompt, return_tensors='pt').to(device)
  output = model.generate(**inputs, max_new_tokens=100, do_sample=True, top_p=0.9)

  generated_text = tokenizer.decode(output[0][inputs['input_ids'].shape[1]:].cpu())

  match = re.search(r"Correct Answer:\s*([A-E])", generated_text)

  if match:
    NUM_RESPONDED += 1
    chosen_answer = match.group(1)
    if chosen_answer == sample["answer"]:
      NUM_CORRECT += 1

# Optionally, consider inferencing multiple sentences in a batch for faster inference;
# If you choose to batch outputs, make sure the results are the same as with batch=1 (using greedy inference)

In [ ]:
print("Responded %%:", NUM_RESPONDED / NUM_SAMPLES)
print("Accuracy (when responded):", NUM_CORRECT / NUM_RESPONDED)
print("Accuracy (overall):", NUM_CORRECT / NUM_SAMPLES)

if NUM_RESPONDED / NUM_SAMPLES < 0.9:
  print("Something is wrong with the evaluation technique (for 5-shot CoT): the model refuses to answer too many questions.")
  print("Make sure you generate enough tokens that the model can produce a correct answer.")
  print("When in doubt, take a look at the full model output. You can often spot errors there.")

__Task 7 (2 points)__ Experiment time!
<img width=200px src=https://www.evolvefish.com/cdn-cgi/image/quality%3D85/assets/images/Apparel/TShirtsWomenCont/Main/EF-APP-CWT-00068(Main).jpg>

Your final quest is to use the testbench you've just written to answer one of the following questions:

### Option 1: How many shots do you need?

How does model accuracy change with the number of fewshot examples?

a. check if the model accuracy changes as you increase/decrease the number of "shots"

b. try to prompt-engineer a model into giving the best rationale __without__ any few-shot examples, i.e. zero-shot

For zero-shot mode, feel free to use wild prompt-engineering or modify the inference procedure.

### Option 2: Is this prompting tecnique reliable?

_Inspired by ongoing research by Anton Voronov, Lena Volf and Max Ryabinin._

For this option, you need to check if the model behavior (and hence, accuracy) is robust to perturbations in the input prompt.

a. Does the accuracy degrade if you provide wrong answers to few-shot examples? (make sure to modify rationale if it contains answer in the end)

b. Does it degrade if you replace question/answer prompts with "Q" and "A"? What if you write both on the same line? Change few-shot separators?



### Option 3: Inference Matters

There are many ways to inference the model, not all of them equal.

a. check whether greedy inference or beam search affects model generation quality

b. implement and evaluate sampling with voting (see explanation below).


The voting technique(b) should work as follows: first, you generate k (e.g. 50) "attempts" at an answer using nucleus sampling (or a similar technique).
Then, you count how many of those attempts chose a particular option (A, B, etc) as the final answer. The option that was chosen most frequently has the most "votes", and therefore "wins".

To speed up voting, you may want to generate these attempts in parallel as a batch. That should be very easy to implement: just run `model.generate` on a list with multiple copies of the same prompt.




================================================

__Common rules:__ You will need to test both hypothes (A and B) in the chosen option. You may choose to replace one of them with your own idea - but please ask course staff in advance (via telegram) if you want full points.

Feel free to organize your code and report as you see fit - but please make sure it's readable and the code runs top-to-bottom :)
Write a short informal report about what you tried and, in doing so, what did you found. Minimum of 2 paragraphs; more is ok; creative visualizations are welcome.

You are allowed (but not required) to prompt the model into generating a report for you --- or helping you write one. However, if you do so, make sure that it is still human-readable :)



In [ ]:
# feel free to organize your solution as you see it

